# Chapter 2: Opening and Reading NetCDF files using STAC
Creators: Ann Windnagel and Robyn Marowitz

Affiliation: [National Snow And Ice Data Center](https://nsidc.org/home)

## Learning Objectives

In this notebook we will learn how to open and read NetCDF files using STAC. While we do this we will explore the features and vairables of the STAC Collection and Items.

### Import relevant packages
If you're interested in learning more ....link package documentation


In [50]:
import json
import requests

import pystac
from netCDF4 import Dataset

## STAC Collections

[A STAC Collection](https://github.com/radiantearth/stac-spec/tree/master/collection-spec) object is used to describe a group of reltaed Items.  

To begin we will assign our `collection.json` to a variable `collection` so that we can explore it using `pystac`.

In [14]:
collection = pystac.Collection.from_file('https://noaadata.apps.nsidc.org/NOAA/G02202_V4/stac/collection.json')

## Explore Collection Information

In the following cells we will look at how the Collection is organized. Using the `describe` method we will see all of the `Items` listed. In the `Item id` there is already a wealth of information - `nh` means northern hemisphere and `sh` means southern hemisphere.

In [4]:
collection.describe()

* <Collection id=noaa-cdr-sea-ice-concentration>
  * <Item id=seaice_conc_daily_nh_1996_v04r00>
  * <Item id=seaice_conc_monthly_nh_197811_202312_v04r00>
  * <Item id=seaice_conc_monthly_sh_197811_202312_v04r00>
  * <Item id=seaice_conc_daily_sh_2023_v04r00>
  * <Item id=seaice_conc_daily_sh_2022_v04r00>
  * <Item id=seaice_conc_daily_sh_2021_v04r00>
  * <Item id=seaice_conc_daily_sh_2020_v04r00>
  * <Item id=seaice_conc_daily_sh_2019_v04r00>
  * <Item id=seaice_conc_daily_sh_2018_v04r00>
  * <Item id=seaice_conc_daily_sh_2017_v04r00>
  * <Item id=seaice_conc_daily_sh_2016_v04r00>
  * <Item id=seaice_conc_daily_sh_2015_v04r00>
  * <Item id=seaice_conc_daily_sh_2014_v04r00>
  * <Item id=seaice_conc_daily_sh_2013_v04r00>
  * <Item id=seaice_conc_daily_sh_2012_v04r00>
  * <Item id=seaice_conc_daily_sh_2011_v04r00>
  * <Item id=seaice_conc_daily_sh_2010_v04r00>
  * <Item id=seaice_conc_daily_sh_2009_v04r00>
  * <Item id=seaice_conc_daily_sh_2008_v04r00>
  * <Item id=seaice_conc_daily_sh_20

In [18]:
print('Title: ',collection.title)
# look into pretty print
print('Description: ',collection.description)

Title:  Sea Ice Concentration CDR
Description:  The Sea Ice Concentration Climate Data Record (CDR) provides a consistent daily and monthly time series of sea ice concentrations for both the north and south Polar Regions on a 25 km x 25 km grid. These data can be used to estimate how much of the ocean surface is covered by ice, and monitor changes in sea ice concentration. The CDR combines concentration estimates using two algorithms developed at the NASA Goddard Space Flight Center (GSFC). Gridded brightness temperatures acquired from a number of Defense Meteorological Satellite Program (DMSP) passive microwave radiometers provide the necessary input to produce the dataset.


## Get all items 

In [21]:
items = list(collection.get_all_items())

print(f"Number of items: {len(items)}")

Number of items: 22


## Explore an item

We will select one item and explore its metadata.

In [23]:
daily_sh_2023_item = collection.get_item("seaice_conc_daily_sh_2023_v04r00", recursive=True)

In [24]:
daily_sh_2023_item.geometry

{'type': 'Polygon',
 'coordinates': [[[180.0, -90.0],
   [180.0, -41.45],
   [-180.0, -41.45],
   [-180.0, -90.0],
   [180.0, -90.0]]]}

In [25]:
daily_sh_2023_item.bbox

[-180.0, -90.0, 180.0, -41.45]

In [33]:
daily_sh_2023_item.properties['start_datetime']


'2023-01-01T00:00:00Z'

In [34]:
daily_sh_2023_item.properties['end_datetime']


'2023-12-31T23:59:59Z'

In [36]:
daily_sh_2023_item.set_datetime(daily_sh_2023_item.properties['start_datetime'])

In [37]:
daily_sh_2023_item.datetime

'2023-01-01T00:00:00Z'

## Read NetCDF

In [43]:
# Access the NetCDF asset and see the components
daily_sh_2023_item.assets['netcdf']

<Asset href=https://noaadata.apps.nsidc.org/NOAA/G02202_V4/south/aggregate/seaice_conc_daily_sh_2023_v04r00.nc>

In [44]:
netcdf_url = daily_sh_2023_item.assets['netcdf'].href
netcdf_url

'https://noaadata.apps.nsidc.org/NOAA/G02202_V4/south/aggregate/seaice_conc_daily_sh_2023_v04r00.nc'

In [49]:
f = Dataset('https://noaadata.apps.nsidc.org/NOAA/G02202_V4/south/aggregate/seaice_conc_daily_sh_2023_v04r00.nc')

syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <html^><head><title>404 Not Found</title></head><body><center><h1>404 Not Found</h1></center><hr><center>nginx/1.23.4</center></body></html>


OSError: [Errno -90] NetCDF: file not found: 'https://noaadata.apps.nsidc.org/NOAA/G02202_V4/south/aggregate/seaice_conc_daily_sh_2023_v04r00.nc'